In [1]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
#import catboost as cb
#import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, ShuffleSplit,StratifiedKFold,TimeSeriesSplit,KFold,GroupKFold,train_test_split,GroupShuffleSplit,StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score,mean_squared_error,mean_absolute_error,log_loss,confusion_matrix,accuracy_score
import sqlite3
#import xgboost as xgb
import datetime
from sklearn.linear_model import LogisticRegression
from scipy.stats import pearsonr
import gc
from sklearn.model_selection import TimeSeriesSplit
#from bayes_opt import BayesianOptimization
import re
from string import punctuation
from scipy.spatial import Voronoi
from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay
from tqdm.notebook import tqdm
#from numba import jit
from collections import Counter
import json
import joblib
import multiprocessing
import time
import keras
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization,Embedding, LSTM, \
                            Bidirectional,GRU,Input,Conv1D,GlobalAveragePooling1D,GlobalMaxPool1D,concatenate,MaxPooling1D
import tensorflow.keras as tk
import math

Using TensorFlow backend.


In [ ]:
Word2Vec()

In [13]:
import logging

In [14]:
# click_log = pd.read_pickle('../raw_data/processed_data/click_log.pkl')
# user =  pd.read_pickle('../raw_data/processed_data/user.pkl')
# ad =  pd.read_pickle('../raw_data/processed_data/ad.pkl')
# click_log_test = pd.read_csv('../../tencent/raw_data/test/click_log.csv')
# ad_test = pd.read_csv('../../tencent/raw_data/test/ad.csv')

In [15]:
# train = click_log.merge(ad,how='left',on = 'creative_id')
# test = click_log_test.merge(ad_test,how='left',on='creative_id')
# train['is_train'] = 1
# test['is_train'] = 0
# all_merge = train.append(test,ignore_index=True)
# for _ in ['industry','product_id']:
#     all_merge[_] = all_merge[_].replace('\\N',-1)
#     all_merge[_] = all_merge[_].astype(np.int64)
# def reduce_mem_usage(df, verbose=True):
#     numerics = ['int16','int32', 'int64', 'float64']
#     start_mem = df.memory_usage().sum() / 1024**2    
#     for col in df.columns:
#         col_type = df[col].dtypes
#         if col_type in numerics:
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if str(col_type)[:3] == 'int':
#                 if c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                     df[col] = df[col].astype(np.int16)
#                 elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                     df[col] = df[col].astype(np.int32)
#                 elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                     df[col] = df[col].astype(np.int64)  
#             else:
#                 if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                     df[col] = df[col].astype(np.float32)
#                 else:
#                     df[col] = df[col].astype(np.float64)    
#     end_mem = df.memory_usage().sum() / 1024**2
#     if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
#     return df
# all_merge = reduce_mem_usage(all_merge)

In [16]:
all_merge = pd.read_pickle('../raw_data/all_merge.pkl')

In [17]:
#### item2vec
def word2vec_feature(prefix, df, groupby, target,size,window,min_count,iter1,filename):
    df_bag = pd.DataFrame(df.sort_values('time')[[groupby, target]])
    df_bag[target] = df_bag[target].astype(str)
    #df_bag[target].fillna('NAN', inplace=True)
    df_bag = df_bag.groupby(groupby, as_index=False)[target].agg({'list':(lambda x: list(set(x)))})
    doc_list = list(df_bag['list'].values)
    logging.basicConfig(
        format='%(asctime)s:%(levelname)s:%(message)s', level=logging.INFO)
    w2v = Word2Vec(doc_list, size=size, window=window, min_count=min_count, workers=32,sg=1,seed = 31,iter=iter1)
    #w2v.save('w2v_file_list_version/'+filename)
    vocab_keys = list(w2v.wv.vocab.keys())
    w2v_array = []
    for v in vocab_keys :
        w2v_array.append(list(w2v.wv[v]))
    df_w2v = pd.DataFrame()
    df_w2v['vocab_keys'] = vocab_keys    
    df_w2v = pd.concat([df_w2v, pd.DataFrame(w2v_array)], axis=1)
    df_w2v.columns = [target] + ['w2v_%s_%s_%d'%(prefix,target,x) for x in range(size)]
    print ('df_w2v:' + str(df_w2v.shape))
    return df_w2v

In [18]:
df_w2v_creative_id = word2vec_feature('d200w50m5ter10',all_merge,'user_id','creative_id',200,50,5,10,'w2v_creative_id_200_50_5_10-set-version.model')
df_w2v_creative_id.to_pickle('../w2v_file_set_version_v3/w2v_creative_id_embedding_200-50-5-10-set-version.pkl')

2020-06-30 23:05:50,624:INFO:collecting all words and their counts
2020-06-30 23:05:50,624:INFO:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-06-30 23:05:50,757:INFO:PROGRESS: at sentence #10000, processed 307484 words, keeping 150228 word types
2020-06-30 23:05:50,901:INFO:PROGRESS: at sentence #20000, processed 617972 words, keeping 252881 word types
2020-06-30 23:05:51,051:INFO:PROGRESS: at sentence #30000, processed 928652 words, keeping 337989 word types
2020-06-30 23:05:51,224:INFO:PROGRESS: at sentence #40000, processed 1233555 words, keeping 411658 word types
2020-06-30 23:05:51,391:INFO:PROGRESS: at sentence #50000, processed 1544657 words, keeping 480886 word types
2020-06-30 23:05:51,554:INFO:PROGRESS: at sentence #60000, processed 1852534 words, keeping 542999 word types
2020-06-30 23:05:51,722:INFO:PROGRESS: at sentence #70000, processed 2162961 words, keeping 601314 word types
2020-06-30 23:05:51,892:INFO:PROGRESS: at sentence #80000, processed 24

2020-06-30 23:06:16,267:INFO:PROGRESS: at sentence #1440000, processed 44190511 words, keeping 3046424 word types
2020-06-30 23:06:16,446:INFO:PROGRESS: at sentence #1450000, processed 44497174 words, keeping 3055245 word types
2020-06-30 23:06:16,623:INFO:PROGRESS: at sentence #1460000, processed 44801297 words, keeping 3063727 word types
2020-06-30 23:06:16,805:INFO:PROGRESS: at sentence #1470000, processed 45112904 words, keeping 3072391 word types
2020-06-30 23:06:16,986:INFO:PROGRESS: at sentence #1480000, processed 45421448 words, keeping 3080840 word types
2020-06-30 23:06:17,164:INFO:PROGRESS: at sentence #1490000, processed 45726024 words, keeping 3089271 word types
2020-06-30 23:06:17,344:INFO:PROGRESS: at sentence #1500000, processed 46034413 words, keeping 3097930 word types
2020-06-30 23:06:17,525:INFO:PROGRESS: at sentence #1510000, processed 46340687 words, keeping 3106147 word types
2020-06-30 23:06:17,699:INFO:PROGRESS: at sentence #1520000, processed 46642405 words, k

2020-06-30 23:06:29,059:INFO:PROGRESS: at sentence #2160000, processed 66192567 words, keeping 3573182 word types
2020-06-30 23:06:29,232:INFO:PROGRESS: at sentence #2170000, processed 66492503 words, keeping 3579565 word types
2020-06-30 23:06:29,407:INFO:PROGRESS: at sentence #2180000, processed 66801157 words, keeping 3585855 word types
2020-06-30 23:06:29,582:INFO:PROGRESS: at sentence #2190000, processed 67108192 words, keeping 3592227 word types
2020-06-30 23:06:29,756:INFO:PROGRESS: at sentence #2200000, processed 67412495 words, keeping 3598513 word types
2020-06-30 23:06:29,931:INFO:PROGRESS: at sentence #2210000, processed 67714907 words, keeping 3604532 word types
2020-06-30 23:06:30,106:INFO:PROGRESS: at sentence #2220000, processed 68018739 words, keeping 3610524 word types
2020-06-30 23:06:30,284:INFO:PROGRESS: at sentence #2230000, processed 68328059 words, keeping 3616792 word types
2020-06-30 23:06:30,462:INFO:PROGRESS: at sentence #2240000, processed 68639084 words, k

2020-06-30 23:06:54,500:INFO:PROGRESS: at sentence #3580000, processed 109842213 words, keeping 4283802 word types
2020-06-30 23:06:54,686:INFO:PROGRESS: at sentence #3590000, processed 110152582 words, keeping 4287805 word types
2020-06-30 23:06:54,869:INFO:PROGRESS: at sentence #3600000, processed 110458350 words, keeping 4291985 word types
2020-06-30 23:06:55,053:INFO:PROGRESS: at sentence #3610000, processed 110764173 words, keeping 4295963 word types
2020-06-30 23:06:55,236:INFO:PROGRESS: at sentence #3620000, processed 111068237 words, keeping 4299856 word types
2020-06-30 23:06:55,421:INFO:PROGRESS: at sentence #3630000, processed 111375113 words, keeping 4303718 word types
2020-06-30 23:06:55,602:INFO:PROGRESS: at sentence #3640000, processed 111678262 words, keeping 4307609 word types
2020-06-30 23:06:55,828:INFO:PROGRESS: at sentence #3650000, processed 112046595 words, keeping 4315108 word types
2020-06-30 23:06:56,013:INFO:PROGRESS: at sentence #3660000, processed 112349924

2020-06-30 23:12:37,584:INFO:EPOCH 1 - PROGRESS: at 2.01% examples, 84014 words/s, in_qsize 64, out_qsize 0
2020-06-30 23:12:38,693:INFO:EPOCH 1 - PROGRESS: at 2.07% examples, 83449 words/s, in_qsize 64, out_qsize 0
2020-06-30 23:12:39,772:INFO:EPOCH 1 - PROGRESS: at 2.19% examples, 84887 words/s, in_qsize 64, out_qsize 0
2020-06-30 23:12:40,869:INFO:EPOCH 1 - PROGRESS: at 2.25% examples, 84365 words/s, in_qsize 64, out_qsize 0
2020-06-30 23:12:41,965:INFO:EPOCH 1 - PROGRESS: at 2.32% examples, 83879 words/s, in_qsize 64, out_qsize 1
2020-06-30 23:12:43,023:INFO:EPOCH 1 - PROGRESS: at 2.44% examples, 85494 words/s, in_qsize 63, out_qsize 0
2020-06-30 23:12:44,033:INFO:EPOCH 1 - PROGRESS: at 2.50% examples, 85208 words/s, in_qsize 64, out_qsize 0
2020-06-30 23:12:45,197:INFO:EPOCH 1 - PROGRESS: at 2.57% examples, 84843 words/s, in_qsize 64, out_qsize 0
2020-06-30 23:12:46,252:INFO:EPOCH 1 - PROGRESS: at 2.68% examples, 85526 words/s, in_qsize 64, out_qsize 0
2020-06-30 23:12:47,326:INFO

...

2020-07-01 02:41:38,913:INFO:EPOCH 10 - PROGRESS: at 78.68% examples, 91386 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:41:40,118:INFO:EPOCH 10 - PROGRESS: at 78.75% examples, 91353 words/s, in_qsize 64, out_qsize 0
2020-07-01 02:41:41,125:INFO:EPOCH 10 - PROGRESS: at 78.86% examples, 91395 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:41:42,138:INFO:EPOCH 10 - PROGRESS: at 78.93% examples, 91388 words/s, in_qsize 64, out_qsize 0
2020-07-01 02:41:43,211:INFO:EPOCH 10 - PROGRESS: at 78.99% examples, 91357 words/s, in_qsize 64, out_qsize 0
2020-07-01 02:41:44,419:INFO:EPOCH 10 - PROGRESS: at 79.12% examples, 91399 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:41:45,549:INFO:EPOCH 10 - PROGRESS: at 79.20% examples, 91392 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:41:46,594:INFO:EPOCH 10 - PROGRESS: at 79.25% examples, 91355 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:41:47,632:INFO:EPOCH 10 - PROGRESS: at 79.36% examples, 91392 words/s, in_qsize 64, out_qsize 0
2020-07-01

2020-07-01 02:43:01,338:INFO:EPOCH 10 - PROGRESS: at 85.06% examples, 91365 words/s, in_qsize 64, out_qsize 0
2020-07-01 02:43:02,365:INFO:EPOCH 10 - PROGRESS: at 85.16% examples, 91384 words/s, in_qsize 64, out_qsize 0
2020-07-01 02:43:03,486:INFO:EPOCH 10 - PROGRESS: at 85.24% examples, 91378 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:43:04,605:INFO:EPOCH 10 - PROGRESS: at 85.31% examples, 91363 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:43:05,611:INFO:EPOCH 10 - PROGRESS: at 85.41% examples, 91384 words/s, in_qsize 64, out_qsize 0
2020-07-01 02:43:06,850:INFO:EPOCH 10 - PROGRESS: at 85.50% examples, 91377 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:43:07,915:INFO:EPOCH 10 - PROGRESS: at 85.57% examples, 91358 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:43:09,014:INFO:EPOCH 10 - PROGRESS: at 85.69% examples, 91397 words/s, in_qsize 64, out_qsize 0
2020-07-01 02:43:10,190:INFO:EPOCH 10 - PROGRESS: at 85.76% examples, 91377 words/s, in_qsize 64, out_qsize 0
2020-07-01

2020-07-01 02:44:24,124:INFO:EPOCH 10 - PROGRESS: at 91.48% examples, 91355 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:44:25,171:INFO:EPOCH 10 - PROGRESS: at 91.59% examples, 91388 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:44:26,264:INFO:EPOCH 10 - PROGRESS: at 91.67% examples, 91384 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:44:27,490:INFO:EPOCH 10 - PROGRESS: at 91.73% examples, 91353 words/s, in_qsize 64, out_qsize 0
2020-07-01 02:44:28,526:INFO:EPOCH 10 - PROGRESS: at 91.85% examples, 91386 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:44:29,585:INFO:EPOCH 10 - PROGRESS: at 91.93% examples, 91386 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:44:30,713:INFO:EPOCH 10 - PROGRESS: at 91.99% examples, 91355 words/s, in_qsize 64, out_qsize 0
2020-07-01 02:44:31,713:INFO:EPOCH 10 - PROGRESS: at 92.10% examples, 91391 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:44:32,736:INFO:EPOCH 10 - PROGRESS: at 92.16% examples, 91368 words/s, in_qsize 64, out_qsize 3
2020-07-01

2020-07-01 02:45:46,531:INFO:EPOCH 10 - PROGRESS: at 97.88% examples, 91358 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:45:47,661:INFO:EPOCH 10 - PROGRESS: at 98.01% examples, 91397 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:45:48,713:INFO:EPOCH 10 - PROGRESS: at 98.10% examples, 91397 words/s, in_qsize 64, out_qsize 0
2020-07-01 02:45:49,868:INFO:EPOCH 10 - PROGRESS: at 98.15% examples, 91366 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:45:50,876:INFO:EPOCH 10 - PROGRESS: at 98.28% examples, 91407 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:45:51,877:INFO:EPOCH 10 - PROGRESS: at 98.34% examples, 91395 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:45:52,923:INFO:EPOCH 10 - PROGRESS: at 98.40% examples, 91373 words/s, in_qsize 64, out_qsize 0
2020-07-01 02:45:53,943:INFO:EPOCH 10 - PROGRESS: at 98.51% examples, 91397 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:45:55,078:INFO:EPOCH 10 - PROGRESS: at 98.59% examples, 91391 words/s, in_qsize 64, out_qsize 0
2020-07-01

df_w2v:(1643245, 201)


In [19]:
df_w2v_ad_id = word2vec_feature('d200w50m5ter10',all_merge,'user_id','ad_id',200,50,5,10,'w2v_ad_id_200_50_5_10-set-version.model')
df_w2v_ad_id.to_pickle('../w2v_file_set_version_v3/w2v_ad_id_embedding_200-50-5-10-set-version.pkl')

2020-07-01 02:52:42,985:INFO:collecting all words and their counts
2020-07-01 02:52:42,986:INFO:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-07-01 02:52:43,117:INFO:PROGRESS: at sentence #10000, processed 306942 words, keeping 147072 word types
2020-07-01 02:52:43,264:INFO:PROGRESS: at sentence #20000, processed 616930 words, keeping 246581 word types
2020-07-01 02:52:43,413:INFO:PROGRESS: at sentence #30000, processed 927055 words, keeping 328447 word types
2020-07-01 02:52:43,586:INFO:PROGRESS: at sentence #40000, processed 1231435 words, keeping 398937 word types
2020-07-01 02:52:43,751:INFO:PROGRESS: at sentence #50000, processed 1541991 words, keeping 465034 word types
2020-07-01 02:52:43,916:INFO:PROGRESS: at sentence #60000, processed 1849310 words, keeping 524211 word types
2020-07-01 02:52:44,085:INFO:PROGRESS: at sentence #70000, processed 2159174 words, keeping 579405 word types
2020-07-01 02:52:44,259:INFO:PROGRESS: at sentence #80000, processed 24

2020-07-01 02:53:08,602:INFO:PROGRESS: at sentence #1440000, processed 44114495 words, keeping 2733395 word types
2020-07-01 02:53:08,784:INFO:PROGRESS: at sentence #1450000, processed 44420619 words, keeping 2740538 word types
2020-07-01 02:53:08,966:INFO:PROGRESS: at sentence #1460000, processed 44724251 words, keeping 2747343 word types
2020-07-01 02:53:09,152:INFO:PROGRESS: at sentence #1470000, processed 45035335 words, keeping 2754391 word types
2020-07-01 02:53:09,336:INFO:PROGRESS: at sentence #1480000, processed 45343376 words, keeping 2761227 word types
2020-07-01 02:53:09,518:INFO:PROGRESS: at sentence #1490000, processed 45647459 words, keeping 2768007 word types
2020-07-01 02:53:09,701:INFO:PROGRESS: at sentence #1500000, processed 45955339 words, keeping 2775033 word types
2020-07-01 02:53:09,883:INFO:PROGRESS: at sentence #1510000, processed 46261042 words, keeping 2781704 word types
2020-07-01 02:53:10,064:INFO:PROGRESS: at sentence #1520000, processed 46562260 words, k

2020-07-01 02:53:21,676:INFO:PROGRESS: at sentence #2160000, processed 66079221 words, keeping 3153314 word types
2020-07-01 02:53:21,848:INFO:PROGRESS: at sentence #2170000, processed 66378611 words, keeping 3158321 word types
2020-07-01 02:53:22,025:INFO:PROGRESS: at sentence #2180000, processed 66686714 words, keeping 3163173 word types
2020-07-01 02:53:22,201:INFO:PROGRESS: at sentence #2190000, processed 66993230 words, keeping 3168213 word types
2020-07-01 02:53:22,376:INFO:PROGRESS: at sentence #2200000, processed 67297027 words, keeping 3173154 word types
2020-07-01 02:53:22,550:INFO:PROGRESS: at sentence #2210000, processed 67598940 words, keeping 3177878 word types
2020-07-01 02:53:22,724:INFO:PROGRESS: at sentence #2220000, processed 67902223 words, keeping 3182548 word types
2020-07-01 02:53:22,901:INFO:PROGRESS: at sentence #2230000, processed 68210964 words, keeping 3187458 word types
2020-07-01 02:53:23,079:INFO:PROGRESS: at sentence #2240000, processed 68521382 words, k

2020-07-01 02:53:47,134:INFO:PROGRESS: at sentence #3580000, processed 109652492 words, keeping 3693819 word types
2020-07-01 02:53:47,318:INFO:PROGRESS: at sentence #3590000, processed 109962314 words, keeping 3696756 word types
2020-07-01 02:53:47,503:INFO:PROGRESS: at sentence #3600000, processed 110267534 words, keeping 3699846 word types
2020-07-01 02:53:47,686:INFO:PROGRESS: at sentence #3610000, processed 110572816 words, keeping 3702737 word types
2020-07-01 02:53:47,869:INFO:PROGRESS: at sentence #3620000, processed 110876337 words, keeping 3705597 word types
2020-07-01 02:53:48,054:INFO:PROGRESS: at sentence #3630000, processed 111182729 words, keeping 3708353 word types
2020-07-01 02:53:48,236:INFO:PROGRESS: at sentence #3640000, processed 111485370 words, keeping 3711195 word types
2020-07-01 02:53:48,463:INFO:PROGRESS: at sentence #3650000, processed 111851632 words, keeping 3716842 word types
2020-07-01 02:53:48,647:INFO:PROGRESS: at sentence #3660000, processed 112154470

2020-07-01 02:59:31,662:INFO:EPOCH 1 - PROGRESS: at 2.04% examples, 84159 words/s, in_qsize 64, out_qsize 0
2020-07-01 02:59:32,706:INFO:EPOCH 1 - PROGRESS: at 2.13% examples, 85082 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:59:33,879:INFO:EPOCH 1 - PROGRESS: at 2.22% examples, 84972 words/s, in_qsize 62, out_qsize 1
2020-07-01 02:59:34,980:INFO:EPOCH 1 - PROGRESS: at 2.29% examples, 84758 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:59:36,036:INFO:EPOCH 1 - PROGRESS: at 2.39% examples, 85549 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:59:37,110:INFO:EPOCH 1 - PROGRESS: at 2.47% examples, 85674 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:59:38,167:INFO:EPOCH 1 - PROGRESS: at 2.53% examples, 85284 words/s, in_qsize 64, out_qsize 0
2020-07-01 02:59:39,191:INFO:EPOCH 1 - PROGRESS: at 2.62% examples, 85803 words/s, in_qsize 63, out_qsize 0
2020-07-01 02:59:40,260:INFO:EPOCH 1 - PROGRESS: at 2.69% examples, 85402 words/s, in_qsize 61, out_qsize 2
2020-07-01 02:59:41,279:INFO

...

2020-07-01 06:30:21,002:INFO:EPOCH 10 - PROGRESS: at 80.70% examples, 90800 words/s, in_qsize 64, out_qsize 1
2020-07-01 06:30:22,022:INFO:EPOCH 10 - PROGRESS: at 80.80% examples, 90822 words/s, in_qsize 64, out_qsize 0
2020-07-01 06:30:23,038:INFO:EPOCH 10 - PROGRESS: at 80.88% examples, 90816 words/s, in_qsize 64, out_qsize 0
2020-07-01 06:30:24,114:INFO:EPOCH 10 - PROGRESS: at 80.94% examples, 90796 words/s, in_qsize 63, out_qsize 0
2020-07-01 06:30:25,250:INFO:EPOCH 10 - PROGRESS: at 81.05% examples, 90817 words/s, in_qsize 64, out_qsize 0
2020-07-01 06:30:26,296:INFO:EPOCH 10 - PROGRESS: at 81.11% examples, 90800 words/s, in_qsize 64, out_qsize 0
2020-07-01 06:30:27,533:INFO:EPOCH 10 - PROGRESS: at 81.20% examples, 90794 words/s, in_qsize 63, out_qsize 0
2020-07-01 06:30:28,654:INFO:EPOCH 10 - PROGRESS: at 81.31% examples, 90826 words/s, in_qsize 64, out_qsize 0
2020-07-01 06:30:29,876:INFO:EPOCH 10 - PROGRESS: at 81.39% examples, 90803 words/s, in_qsize 63, out_qsize 0
2020-07-01

2020-07-01 06:31:43,564:INFO:EPOCH 10 - PROGRESS: at 87.07% examples, 90821 words/s, in_qsize 63, out_qsize 0
2020-07-01 06:31:44,633:INFO:EPOCH 10 - PROGRESS: at 87.14% examples, 90804 words/s, in_qsize 64, out_qsize 0
2020-07-01 06:31:45,653:INFO:EPOCH 10 - PROGRESS: at 87.22% examples, 90807 words/s, in_qsize 64, out_qsize 0
2020-07-01 06:31:47,077:INFO:EPOCH 10 - PROGRESS: at 87.32% examples, 90804 words/s, in_qsize 63, out_qsize 0
2020-07-01 06:31:48,101:INFO:EPOCH 10 - PROGRESS: at 87.42% examples, 90823 words/s, in_qsize 64, out_qsize 0
2020-07-01 06:31:49,120:INFO:EPOCH 10 - PROGRESS: at 87.48% examples, 90809 words/s, in_qsize 63, out_qsize 0
2020-07-01 06:31:50,362:INFO:EPOCH 10 - PROGRESS: at 87.58% examples, 90812 words/s, in_qsize 63, out_qsize 0
2020-07-01 06:31:51,427:INFO:EPOCH 10 - PROGRESS: at 87.67% examples, 90820 words/s, in_qsize 62, out_qsize 1
2020-07-01 06:31:52,650:INFO:EPOCH 10 - PROGRESS: at 87.75% examples, 90807 words/s, in_qsize 64, out_qsize 1
2020-07-01

2020-07-01 06:33:05,730:INFO:EPOCH 10 - PROGRESS: at 93.36% examples, 90811 words/s, in_qsize 64, out_qsize 0
2020-07-01 06:33:06,856:INFO:EPOCH 10 - PROGRESS: at 93.43% examples, 90798 words/s, in_qsize 63, out_qsize 0
2020-07-01 06:33:08,207:INFO:EPOCH 10 - PROGRESS: at 93.53% examples, 90792 words/s, in_qsize 63, out_qsize 0
2020-07-01 06:33:09,269:INFO:EPOCH 10 - PROGRESS: at 93.62% examples, 90800 words/s, in_qsize 63, out_qsize 0
2020-07-01 06:33:10,531:INFO:EPOCH 10 - PROGRESS: at 93.70% examples, 90793 words/s, in_qsize 62, out_qsize 2
2020-07-01 06:33:11,584:INFO:EPOCH 10 - PROGRESS: at 93.80% examples, 90809 words/s, in_qsize 64, out_qsize 0
2020-07-01 06:33:12,677:INFO:EPOCH 10 - PROGRESS: at 93.86% examples, 90783 words/s, in_qsize 64, out_qsize 0
2020-07-01 06:33:13,750:INFO:EPOCH 10 - PROGRESS: at 93.96% examples, 90806 words/s, in_qsize 63, out_qsize 0
2020-07-01 06:33:14,934:INFO:EPOCH 10 - PROGRESS: at 94.05% examples, 90805 words/s, in_qsize 64, out_qsize 0
2020-07-01

2020-07-01 06:34:27,329:INFO:EPOCH 10 - PROGRESS: at 99.65% examples, 90815 words/s, in_qsize 45, out_qsize 0
2020-07-01 06:34:28,554:INFO:EPOCH 10 - PROGRESS: at 99.71% examples, 90788 words/s, in_qsize 37, out_qsize 0
2020-07-01 06:34:28,815:INFO:worker thread finished; awaiting finish of 31 more threads
2020-07-01 06:34:28,945:INFO:worker thread finished; awaiting finish of 30 more threads
2020-07-01 06:34:29,014:INFO:worker thread finished; awaiting finish of 29 more threads
2020-07-01 06:34:29,096:INFO:worker thread finished; awaiting finish of 28 more threads
2020-07-01 06:34:29,107:INFO:worker thread finished; awaiting finish of 27 more threads
2020-07-01 06:34:29,152:INFO:worker thread finished; awaiting finish of 26 more threads
2020-07-01 06:34:29,316:INFO:worker thread finished; awaiting finish of 25 more threads
2020-07-01 06:34:29,402:INFO:worker thread finished; awaiting finish of 24 more threads
2020-07-01 06:34:29,411:INFO:worker thread finished; awaiting finish of 23 m

df_w2v:(1594938, 201)


In [20]:
df_w2v_advertiser_id = word2vec_feature('d128w50m5ter10',all_merge,'user_id','advertiser_id',128,50,5,10,'w2v_advertiser_id_128_50_5_10-set-version.model')
df_w2v_advertiser_id.to_pickle('../w2v_file_set_version_v3/w2v_advertiser_id_embedding_128-50-5-10-set-version.pkl')

2020-07-01 06:40:49,713:INFO:collecting all words and their counts
2020-07-01 06:40:49,714:INFO:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-07-01 06:40:49,795:INFO:PROGRESS: at sentence #10000, processed 234399 words, keeping 16253 word types
2020-07-01 06:40:49,875:INFO:PROGRESS: at sentence #20000, processed 472136 words, keeping 20419 word types
2020-07-01 06:40:49,956:INFO:PROGRESS: at sentence #30000, processed 709381 words, keeping 23071 word types
2020-07-01 06:40:50,040:INFO:PROGRESS: at sentence #40000, processed 942776 words, keeping 25255 word types
2020-07-01 06:40:50,124:INFO:PROGRESS: at sentence #50000, processed 1180644 words, keeping 27044 word types
2020-07-01 06:40:50,207:INFO:PROGRESS: at sentence #60000, processed 1416403 words, keeping 28464 word types
2020-07-01 06:40:50,290:INFO:PROGRESS: at sentence #70000, processed 1654096 words, keeping 29702 word types
2020-07-01 06:40:50,373:INFO:PROGRESS: at sentence #80000, processed 1890426 wo

2020-07-01 06:40:55,498:INFO:PROGRESS: at sentence #740000, processed 17383384 words, keeping 50477 word types
2020-07-01 06:40:55,576:INFO:PROGRESS: at sentence #750000, processed 17616989 words, keeping 50566 word types
2020-07-01 06:40:55,654:INFO:PROGRESS: at sentence #760000, processed 17850973 words, keeping 50668 word types
2020-07-01 06:40:55,733:INFO:PROGRESS: at sentence #770000, processed 18085616 words, keeping 50762 word types
2020-07-01 06:40:55,812:INFO:PROGRESS: at sentence #780000, processed 18321995 words, keeping 50864 word types
2020-07-01 06:40:55,890:INFO:PROGRESS: at sentence #790000, processed 18556249 words, keeping 50958 word types
2020-07-01 06:40:55,969:INFO:PROGRESS: at sentence #800000, processed 18790345 words, keeping 51060 word types
2020-07-01 06:40:56,047:INFO:PROGRESS: at sentence #810000, processed 19027103 words, keeping 51156 word types
2020-07-01 06:40:56,124:INFO:PROGRESS: at sentence #820000, processed 19260092 words, keeping 51251 word types
2

2020-07-01 06:41:01,000:INFO:PROGRESS: at sentence #1480000, processed 34760165 words, keeping 55983 word types
2020-07-01 06:41:01,075:INFO:PROGRESS: at sentence #1490000, processed 34993898 words, keeping 56019 word types
2020-07-01 06:41:01,149:INFO:PROGRESS: at sentence #1500000, processed 35230644 words, keeping 56080 word types
2020-07-01 06:41:01,223:INFO:PROGRESS: at sentence #1510000, processed 35465188 words, keeping 56126 word types
2020-07-01 06:41:01,297:INFO:PROGRESS: at sentence #1520000, processed 35696878 words, keeping 56177 word types
2020-07-01 06:41:01,371:INFO:PROGRESS: at sentence #1530000, processed 35931412 words, keeping 56220 word types
2020-07-01 06:41:01,446:INFO:PROGRESS: at sentence #1540000, processed 36168085 words, keeping 56262 word types
2020-07-01 06:41:01,520:INFO:PROGRESS: at sentence #1550000, processed 36401950 words, keeping 56318 word types
2020-07-01 06:41:01,593:INFO:PROGRESS: at sentence #1560000, processed 36633030 words, keeping 56362 wor

2020-07-01 06:41:06,434:INFO:PROGRESS: at sentence #2220000, processed 52067373 words, keeping 58876 word types
2020-07-01 06:41:06,505:INFO:PROGRESS: at sentence #2230000, processed 52304111 words, keeping 58897 word types
2020-07-01 06:41:06,577:INFO:PROGRESS: at sentence #2240000, processed 52541252 words, keeping 58935 word types
2020-07-01 06:41:06,648:INFO:PROGRESS: at sentence #2250000, processed 52775902 words, keeping 58975 word types
2020-07-01 06:41:06,720:INFO:PROGRESS: at sentence #2260000, processed 53009223 words, keeping 58996 word types
2020-07-01 06:41:06,793:INFO:PROGRESS: at sentence #2270000, processed 53247570 words, keeping 59020 word types
2020-07-01 06:41:06,864:INFO:PROGRESS: at sentence #2280000, processed 53481113 words, keeping 59052 word types
2020-07-01 06:41:06,937:INFO:PROGRESS: at sentence #2290000, processed 53721850 words, keeping 59081 word types
2020-07-01 06:41:07,010:INFO:PROGRESS: at sentence #2300000, processed 53957836 words, keeping 59113 wor

2020-07-01 06:41:11,714:INFO:PROGRESS: at sentence #2960000, processed 69456920 words, keeping 60843 word types
2020-07-01 06:41:11,786:INFO:PROGRESS: at sentence #2970000, processed 69691020 words, keeping 60868 word types
2020-07-01 06:41:11,857:INFO:PROGRESS: at sentence #2980000, processed 69925285 words, keeping 60898 word types
2020-07-01 06:41:11,928:INFO:PROGRESS: at sentence #2990000, processed 70158229 words, keeping 60914 word types
2020-07-01 06:41:12,002:INFO:PROGRESS: at sentence #3000000, processed 70395398 words, keeping 60948 word types
2020-07-01 06:41:12,076:INFO:PROGRESS: at sentence #3010000, processed 70627934 words, keeping 60975 word types
2020-07-01 06:41:12,149:INFO:PROGRESS: at sentence #3020000, processed 70860725 words, keeping 61014 word types
2020-07-01 06:41:12,224:INFO:PROGRESS: at sentence #3030000, processed 71094967 words, keeping 61038 word types
2020-07-01 06:41:12,298:INFO:PROGRESS: at sentence #3040000, processed 71329934 words, keeping 61056 wor

...

2020-07-01 07:28:11,068:INFO:EPOCH 10 - PROGRESS: at 16.80% examples, 283879 words/s, in_qsize 62, out_qsize 1
2020-07-01 07:28:12,077:INFO:EPOCH 10 - PROGRESS: at 17.09% examples, 283269 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:28:13,122:INFO:EPOCH 10 - PROGRESS: at 17.40% examples, 283021 words/s, in_qsize 63, out_qsize 2
2020-07-01 07:28:14,123:INFO:EPOCH 10 - PROGRESS: at 17.74% examples, 283332 words/s, in_qsize 62, out_qsize 1
2020-07-01 07:28:15,127:INFO:EPOCH 10 - PROGRESS: at 18.06% examples, 283288 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:28:16,151:INFO:EPOCH 10 - PROGRESS: at 18.39% examples, 283319 words/s, in_qsize 61, out_qsize 2
2020-07-01 07:28:17,153:INFO:EPOCH 10 - PROGRESS: at 18.73% examples, 283609 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:28:18,217:INFO:EPOCH 10 - PROGRESS: at 19.06% examples, 283433 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:28:19,253:INFO:EPOCH 10 - PROGRESS: at 19.40% examples, 283552 words/s, in_qsize 62, out_qsize 1
2

2020-07-01 07:29:26,991:INFO:EPOCH 10 - PROGRESS: at 41.48% examples, 285897 words/s, in_qsize 64, out_qsize 1
2020-07-01 07:29:27,999:INFO:EPOCH 10 - PROGRESS: at 41.82% examples, 285992 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:29:29,010:INFO:EPOCH 10 - PROGRESS: at 42.16% examples, 286086 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:29:30,047:INFO:EPOCH 10 - PROGRESS: at 42.50% examples, 286044 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:29:31,054:INFO:EPOCH 10 - PROGRESS: at 42.81% examples, 285998 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:29:32,062:INFO:EPOCH 10 - PROGRESS: at 43.14% examples, 286021 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:29:33,087:INFO:EPOCH 10 - PROGRESS: at 43.48% examples, 286081 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:29:34,134:INFO:EPOCH 10 - PROGRESS: at 43.82% examples, 286019 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:29:35,156:INFO:EPOCH 10 - PROGRESS: at 44.16% examples, 286084 words/s, in_qsize 63, out_qsize 0
2

2020-07-01 07:30:43,205:INFO:EPOCH 10 - PROGRESS: at 66.28% examples, 286441 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:30:44,231:INFO:EPOCH 10 - PROGRESS: at 66.57% examples, 286293 words/s, in_qsize 61, out_qsize 2
2020-07-01 07:30:45,248:INFO:EPOCH 10 - PROGRESS: at 66.91% examples, 286339 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:30:46,282:INFO:EPOCH 10 - PROGRESS: at 67.22% examples, 286227 words/s, in_qsize 64, out_qsize 4
2020-07-01 07:30:47,371:INFO:EPOCH 10 - PROGRESS: at 67.58% examples, 286263 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:30:48,388:INFO:EPOCH 10 - PROGRESS: at 67.92% examples, 286309 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:30:49,394:INFO:EPOCH 10 - PROGRESS: at 68.26% examples, 286370 words/s, in_qsize 64, out_qsize 1
2020-07-01 07:30:50,397:INFO:EPOCH 10 - PROGRESS: at 68.60% examples, 286433 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:30:51,442:INFO:EPOCH 10 - PROGRESS: at 68.95% examples, 286437 words/s, in_qsize 64, out_qsize 1
2

2020-07-01 07:31:59,629:INFO:EPOCH 10 - PROGRESS: at 91.04% examples, 286536 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:32:00,642:INFO:EPOCH 10 - PROGRESS: at 91.38% examples, 286573 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:32:01,714:INFO:EPOCH 10 - PROGRESS: at 91.72% examples, 286553 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:32:02,720:INFO:EPOCH 10 - PROGRESS: at 92.04% examples, 286563 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:32:03,756:INFO:EPOCH 10 - PROGRESS: at 92.39% examples, 286577 words/s, in_qsize 64, out_qsize 2
2020-07-01 07:32:04,767:INFO:EPOCH 10 - PROGRESS: at 92.71% examples, 286549 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:32:05,813:INFO:EPOCH 10 - PROGRESS: at 93.05% examples, 286554 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:32:06,826:INFO:EPOCH 10 - PROGRESS: at 93.39% examples, 286589 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:32:07,863:INFO:EPOCH 10 - PROGRESS: at 93.70% examples, 286537 words/s, in_qsize 64, out_qsize 0
2

df_w2v:(47576, 129)


In [21]:
df_w2v_product_id = word2vec_feature('d128w50m5ter10',all_merge,'user_id','product_id',128,50,5,10,'w2v_product_id_128_50_5_10-set-version.model')
df_w2v_product_id.to_pickle('../w2v_file_set_version_v3/w2v_product_id_embedding_128-50-5-10-set-version.pkl')

2020-07-01 07:36:36,387:INFO:collecting all words and their counts
2020-07-01 07:36:36,388:INFO:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-07-01 07:36:36,431:INFO:PROGRESS: at sentence #10000, processed 121281 words, keeping 6404 word types
2020-07-01 07:36:36,473:INFO:PROGRESS: at sentence #20000, processed 244869 words, keeping 8709 word types
2020-07-01 07:36:36,513:INFO:PROGRESS: at sentence #30000, processed 368740 words, keeping 10256 word types
2020-07-01 07:36:36,555:INFO:PROGRESS: at sentence #40000, processed 490406 words, keeping 11706 word types
2020-07-01 07:36:36,598:INFO:PROGRESS: at sentence #50000, processed 615007 words, keeping 12879 word types
2020-07-01 07:36:36,639:INFO:PROGRESS: at sentence #60000, processed 737240 words, keeping 13840 word types
2020-07-01 07:36:36,681:INFO:PROGRESS: at sentence #70000, processed 860131 words, keeping 14683 word types
2020-07-01 07:36:36,723:INFO:PROGRESS: at sentence #80000, processed 982959 words, k

2020-07-01 07:36:39,372:INFO:PROGRESS: at sentence #750000, processed 9181278 words, keeping 31803 word types
2020-07-01 07:36:39,412:INFO:PROGRESS: at sentence #760000, processed 9303057 words, keeping 31896 word types
2020-07-01 07:36:39,452:INFO:PROGRESS: at sentence #770000, processed 9425722 words, keeping 31983 word types
2020-07-01 07:36:39,492:INFO:PROGRESS: at sentence #780000, processed 9549311 words, keeping 32107 word types
2020-07-01 07:36:39,533:INFO:PROGRESS: at sentence #790000, processed 9671437 words, keeping 32185 word types
2020-07-01 07:36:39,572:INFO:PROGRESS: at sentence #800000, processed 9794242 words, keeping 32275 word types
2020-07-01 07:36:39,613:INFO:PROGRESS: at sentence #810000, processed 9917723 words, keeping 32375 word types
2020-07-01 07:36:39,652:INFO:PROGRESS: at sentence #820000, processed 10039007 words, keeping 32467 word types
2020-07-01 07:36:39,692:INFO:PROGRESS: at sentence #830000, processed 10159220 words, keeping 32570 word types
2020-07-

2020-07-01 07:36:44,930:INFO:PROGRESS: at sentence #2220000, processed 27139695 words, keeping 39911 word types
2020-07-01 07:36:44,967:INFO:PROGRESS: at sentence #2230000, processed 27263174 words, keeping 39942 word types
2020-07-01 07:36:45,003:INFO:PROGRESS: at sentence #2240000, processed 27385621 words, keeping 39979 word types
2020-07-01 07:36:45,040:INFO:PROGRESS: at sentence #2250000, processed 27508543 words, keeping 40025 word types
2020-07-01 07:36:45,076:INFO:PROGRESS: at sentence #2260000, processed 27631140 words, keeping 40047 word types
2020-07-01 07:36:45,113:INFO:PROGRESS: at sentence #2270000, processed 27755245 words, keeping 40076 word types
2020-07-01 07:36:45,149:INFO:PROGRESS: at sentence #2280000, processed 27877246 words, keeping 40104 word types
2020-07-01 07:36:45,187:INFO:PROGRESS: at sentence #2290000, processed 28003246 words, keeping 40151 word types
2020-07-01 07:36:45,224:INFO:PROGRESS: at sentence #2300000, processed 28125776 words, keeping 40178 wor

2020-07-01 07:36:47,623:INFO:PROGRESS: at sentence #2960000, processed 36197273 words, keeping 42009 word types
2020-07-01 07:36:47,659:INFO:PROGRESS: at sentence #2970000, processed 36319495 words, keeping 42035 word types
2020-07-01 07:36:47,695:INFO:PROGRESS: at sentence #2980000, processed 36440911 words, keeping 42069 word types
2020-07-01 07:36:47,730:INFO:PROGRESS: at sentence #2990000, processed 36562353 words, keeping 42088 word types
2020-07-01 07:36:47,768:INFO:PROGRESS: at sentence #3000000, processed 36686114 words, keeping 42114 word types
2020-07-01 07:36:47,805:INFO:PROGRESS: at sentence #3010000, processed 36806718 words, keeping 42150 word types
2020-07-01 07:36:47,843:INFO:PROGRESS: at sentence #3020000, processed 36927864 words, keeping 42189 word types
2020-07-01 07:36:47,880:INFO:PROGRESS: at sentence #3030000, processed 37050332 words, keeping 42216 word types
2020-07-01 07:36:47,918:INFO:PROGRESS: at sentence #3040000, processed 37172090 words, keeping 42241 wor

2020-07-01 07:36:50,421:INFO:PROGRESS: at sentence #3700000, processed 45259907 words, keeping 43762 word types
2020-07-01 07:36:50,459:INFO:PROGRESS: at sentence #3710000, processed 45381132 words, keeping 43776 word types
2020-07-01 07:36:50,499:INFO:PROGRESS: at sentence #3720000, processed 45503288 words, keeping 43793 word types
2020-07-01 07:36:50,538:INFO:PROGRESS: at sentence #3730000, processed 45626368 words, keeping 43812 word types
2020-07-01 07:36:50,579:INFO:PROGRESS: at sentence #3740000, processed 45750757 words, keeping 43834 word types
2020-07-01 07:36:50,619:INFO:PROGRESS: at sentence #3750000, processed 45873641 words, keeping 43856 word types
2020-07-01 07:36:50,658:INFO:PROGRESS: at sentence #3760000, processed 45996231 words, keeping 43877 word types
2020-07-01 07:36:50,698:INFO:PROGRESS: at sentence #3770000, processed 46118500 words, keeping 43891 word types
2020-07-01 07:36:50,738:INFO:PROGRESS: at sentence #3780000, processed 46241800 words, keeping 43910 wor

...

2020-07-01 07:47:09,038:INFO:EPOCH 8 - PROGRESS: at 23.70% examples, 420294 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:47:10,045:INFO:EPOCH 8 - PROGRESS: at 24.88% examples, 420616 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:47:11,083:INFO:EPOCH 8 - PROGRESS: at 26.00% examples, 419338 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:47:12,101:INFO:EPOCH 8 - PROGRESS: at 27.30% examples, 421397 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:47:13,138:INFO:EPOCH 8 - PROGRESS: at 28.51% examples, 421426 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:47:14,144:INFO:EPOCH 8 - PROGRESS: at 29.76% examples, 422544 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:47:15,144:INFO:EPOCH 8 - PROGRESS: at 30.91% examples, 422262 words/s, in_qsize 62, out_qsize 1
2020-07-01 07:47:16,152:INFO:EPOCH 8 - PROGRESS: at 32.05% examples, 421890 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:47:17,168:INFO:EPOCH 8 - PROGRESS: at 33.26% examples, 422182 words/s, in_qsize 64, out_qsize 0
2020-07-01

2020-07-01 07:48:13,293:INFO:worker thread finished; awaiting finish of 16 more threads
2020-07-01 07:48:13,311:INFO:worker thread finished; awaiting finish of 15 more threads
2020-07-01 07:48:13,316:INFO:worker thread finished; awaiting finish of 14 more threads
2020-07-01 07:48:13,331:INFO:worker thread finished; awaiting finish of 13 more threads
2020-07-01 07:48:13,348:INFO:worker thread finished; awaiting finish of 12 more threads
2020-07-01 07:48:13,361:INFO:worker thread finished; awaiting finish of 11 more threads
2020-07-01 07:48:13,364:INFO:worker thread finished; awaiting finish of 10 more threads
2020-07-01 07:48:13,365:INFO:worker thread finished; awaiting finish of 9 more threads
2020-07-01 07:48:13,379:INFO:worker thread finished; awaiting finish of 8 more threads
2020-07-01 07:48:13,384:INFO:worker thread finished; awaiting finish of 7 more threads
2020-07-01 07:48:13,385:INFO:worker thread finished; awaiting finish of 6 more threads
2020-07-01 07:48:13,385:INFO:worker 

2020-07-01 07:49:15,626:INFO:EPOCH 9 - PROGRESS: at 72.99% examples, 426128 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:49:16,636:INFO:EPOCH 9 - PROGRESS: at 74.21% examples, 426342 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:49:17,655:INFO:EPOCH 9 - PROGRESS: at 75.44% examples, 426504 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:49:18,731:INFO:EPOCH 9 - PROGRESS: at 76.63% examples, 426064 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:49:19,748:INFO:EPOCH 9 - PROGRESS: at 77.88% examples, 426467 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:49:20,763:INFO:EPOCH 9 - PROGRESS: at 79.02% examples, 426214 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:49:21,805:INFO:EPOCH 9 - PROGRESS: at 80.24% examples, 426230 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:49:22,827:INFO:EPOCH 9 - PROGRESS: at 81.45% examples, 426256 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:49:23,830:INFO:EPOCH 9 - PROGRESS: at 82.60% examples, 426200 words/s, in_qsize 64, out_qsize 2
2020-07-01

2020-07-01 07:50:04,737:INFO:EPOCH 10 - PROGRESS: at 30.83% examples, 424102 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:50:05,744:INFO:EPOCH 10 - PROGRESS: at 32.05% examples, 424762 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:50:06,751:INFO:EPOCH 10 - PROGRESS: at 33.26% examples, 425093 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:50:07,766:INFO:EPOCH 10 - PROGRESS: at 34.48% examples, 425563 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:50:08,768:INFO:EPOCH 10 - PROGRESS: at 35.69% examples, 425940 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:50:09,782:INFO:EPOCH 10 - PROGRESS: at 36.74% examples, 424492 words/s, in_qsize 63, out_qsize 0
2020-07-01 07:50:10,854:INFO:EPOCH 10 - PROGRESS: at 38.05% examples, 425099 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:50:11,859:INFO:EPOCH 10 - PROGRESS: at 39.36% examples, 426493 words/s, in_qsize 64, out_qsize 0
2020-07-01 07:50:12,873:INFO:EPOCH 10 - PROGRESS: at 40.55% examples, 426405 words/s, in_qsize 64, out_qsize 0
2

df_w2v:(28678, 129)


In [12]:
df_bag = pd.DataFrame(all_merge.sort_values('time')[['user_id', 'creative_id','ad_id','advertiser_id','product_id','industry']])
df_bag['creative_id'] = df_bag['creative_id'].astype(str)
df_bag['ad_id'] = df_bag['ad_id'].astype(str)
df_bag['advertiser_id'] = df_bag['advertiser_id'].astype(str)
df_bag['product_id'] = df_bag['product_id'].astype(str)
df_bag['industry'] = df_bag['industry'].astype(str)
df_bag = df_bag.groupby('user_id', as_index=False).agg({'creative_id':{'list_creative_id':lambda x:list(x)},
                                                       'ad_id':{'list_ad_id':lambda x:list(x)},
                                                       'advertiser_id':{'list_advertiser_id':lambda x:list(x)},
                                                       'product_id':{'list_product_id':lambda x:list(x)},
                                                       'industry':{'list_industry':lambda x:list(x)}})

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1455: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  return super().aggregate(arg, *args, **kwargs)


In [15]:
df_bag.columns = ['user_id','list_creative_id','list_ad_id','list_advertiser_id','list_product_id','list_industry']

In [16]:
tmp1= df_bag.list_creative_id.map(len)
tmp2= df_bag.list_ad_id.map(len)
tmp3= df_bag.list_advertiser_id.map(len)
tmp4= df_bag.list_product_id.map(len)
tmp5= df_bag.list_industry.map(len)

In [17]:
df_bag.to_pickle('./fit_generator_list/df_bag_generator.pkl')

In [22]:
####生成数据
def data_prepare(df,embed_df,groupby,target,maxlen):
    df_bag = pd.DataFrame(df.sort_values('time')[[groupby, target]])
    df_bag[target] = df_bag[target].astype(str)
    #df_bag[target].fillna('NAN', inplace=True)
    df_bag = df_bag.groupby(groupby, as_index=False)[target].agg({'list':(lambda x: list(set(x)))})
    tokenizer = Tokenizer(lower=False, char_level=False, split=',')
    tokenizer.fit_on_texts(embed_df[target])
    #n_unique = embed_df.shape[0]
    X_train_seq = tokenizer.texts_to_sequences(df_bag[:3000000]['list'])
    X_test_seq = tokenizer.texts_to_sequences(df_bag[3000000:]['list'])
    X_train = pad_sequences(X_train_seq, maxlen=maxlen, value=0)
    X_test = pad_sequences(X_test_seq, maxlen=maxlen, value=0)
    map_index = embed_df[target].to_dict()
    inverse_map_index = dict()
    for key,values in map_index.items():
        inverse_map_index[values] = key
    array_w2v = embed_df.iloc[:,1:].values
    max_feaures = embed_df.shape[0]+1
    embedding_matrix = np.zeros((max_feaures, embed_df.shape[1]-1))
    for word in tqdm(tokenizer.word_index):
    #     if word in df_w2v_creative_id.creative_id:
    #         continue
        embedding_matrix[tokenizer.word_index[word]] = array_w2v[inverse_map_index[word]]
    return X_train,X_test,embedding_matrix

In [ ]:
del df_bag
gc.collect()

In [23]:

X_train_cre,X_test_cre,embed_cre = data_prepare(all_merge,df_w2v_creative_id,'user_id','creative_id',100)
print('creative_id Done')
X_train_ad,X_test_ad,embed_ad = data_prepare(all_merge,df_w2v_ad_id,'user_id','ad_id',100)
print('ad_id Done')
X_train_adv,X_test_adv,embed_adv = data_prepare(all_merge,df_w2v_advertiser_id,'user_id','advertiser_id',50)
print('adv_id Done')
X_train_prod,X_test_prod,embed_prod = data_prepare(all_merge,df_w2v_product_id,'user_id','product_id',50)
print('Product_id Done')
# X_train_industry,X_test_industry,embed_industry = data_prepare(all_merge,df_w2v_industry,'user_id','industry',100)


creative_id Done



ad_id Done



adv_id Done



Product_id Done


In [ ]:
X_train_industry,X_test_industry,embed_industry = data_prepare(all_merge,df_w2v_industry,'user_id','industry',100)

In [24]:
# pd.DataFrame(X_train_cre).to_pickle('./fit_generator_set/X_train_cre.pkl')     
# pd.DataFrame(X_test_cre).to_pickle('./fit_generator_set/X_test_cre.pkl')
pd.DataFrame(embed_cre).to_pickle('../fit_generator_set_v3/embed_cre.pkl')
# pd.DataFrame(X_train_ad).to_pickle('./fit_generator_set/X_train_ad.pkl')
# pd.DataFrame(X_test_ad).to_pickle('./fit_generator_set/X_test_ad.pkl')
pd.DataFrame(embed_ad).to_pickle('../fit_generator_set_v3/embed_ad.pkl')
# pd.DataFrame(X_train_adv).to_pickle('./fit_generator_set/X_train_adv.pkl')
# pd.DataFrame(X_test_adv).to_pickle('./fit_generator_set/X_test_adv.pkl')
pd.DataFrame(embed_adv).to_pickle('../fit_generator_set_v3/embed_adv.pkl')
# pd.DataFrame(X_train_prod).to_pickle('./fit_generator_set/X_train_prod.pkl')
# pd.DataFrame(X_test_prod).to_pickle('./fit_generator_set/X_test_prod.pkl')
pd.DataFrame(embed_prod).to_pickle('../fit_generator_set_v3/embed_prod.pkl')

# pd.DataFrame(X_train_industry).to_pickle('./fit_generator_set/X_train_industry.pkl')
# pd.DataFrame(X_test_industry).to_pickle('./fit_generator_set/X_test_industry.pkl')
# pd.DataFrame(embed_industry).to_pickle('./fit_generator_set/embed_industry.pkl')

In [23]:
pd.DataFrame(embed_prod).to_pickle('./fit_generator_set/embed_prod.pkl')

In [22]:
print(1)

1


In [3]:
test = pd.read_pickle('./fit_generator_set/embed_cre.pkl')

In [10]:
np.sum(test.values.sum(axis=1)==0)

1